In [1]:
import pandas as pd

df = pd.read_csv('Steam_Reviews_1237320_20240621_sonic_frontiers.csv')
df.head()

,SteamId,ProfileURL,ReviewText,Review,ReviewLength(Chars),PlayHours,DatePosted
0,CatDonkey,https://steamcommunity.com/id/CatDonkey/,Get Ian Flynn to write every Sonic game after ...,Recommended,98,39.7 hrs on record,"Posted: 23 November, 2022"
1,Sanic123,https://steamcommunity.com/id/Sanic123/,Who knew getting good writers could make a goo...,Recommended,44,78.0 hrs on record,"Posted: 10 November, 2022"
2,76561198269316017,https://steamcommunity.com/profiles/7656119826...,"They did it, they made a good Sonic game",Recommended,32,37.6 hrs on record,"Posted: 9 November, 2022"
3,Tetsuo9999,https://steamcommunity.com/id/Tetsuo9999/,Who knew that open world games could be fun if...,Recommended,85,7.9 hrs on record,"Posted: 19 November, 2022"
4,martymcfries,https://steamcommunity.com/id/martymcfries/,this game feels like greeting an old friend ba...,Recommended,52,51.9 hrs on record,"Posted: 13 November, 2022"


In [2]:
data = df[['ReviewText']]
data.dropna(inplace=True)
data.head()

/tmp/ipykernel_4358/3511594172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


,ReviewText
0,Get Ian Flynn to write every Sonic game after ...
1,Who knew getting good writers could make a goo...
2,"They did it, they made a good Sonic game"
3,Who knew that open world games could be fun if...
4,this game feels like greeting an old friend ba...


In [3]:
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import TextGeneration
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.preprocessing import normalize
import scipy.sparse as sp

class NormalizedClassTfidfTransformer(ClassTfidfTransformer):
    def transform(self, X):
        # Perform regular c-TF-IDF transformation
        X_transformed = super().transform(X)
        
        # Apply L2 normalization
        X_normalized = normalize(X_transformed, norm='l2', axis=1)
        
        return sp.csr_matrix(X_normalized)

ctfidf_model = NormalizedClassTfidfTransformer()
vectorizer_model = CountVectorizer(stop_words="english")
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, verbose=True)

/home/fiqri/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_array = data.to_numpy()
data_string = []
for x in data_array:
  data_string.append(x[0])
topics, probs = topic_model.fit_transform(data_string)

2024-11-17 15:52:11,287 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 188/188 [00:30<00:00,  6.18it/s]
2024-11-17 15:52:46,543 - BERTopic - Embedding - Completed ✓
2024-11-17 15:52:46,545 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-17 15:53:56,681 - BERTopic - Dimensionality - Completed ✓
2024-11-17 15:53:56,684 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-17 15:53:57,366 - BERTopic - Cluster - Completed ✓
2024-11-17 15:53:57,383 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-17 15:53:58,474 - BERTopic - Representation - Completed ✓


In [16]:
topic_represent = topic_model.get_topic_info()[['Topic', 'Representation']]
dictionary = topic_represent.set_index('Topic')['Representation'].to_dict()
print(dictionary)

{-1: ['game', 'sonic', 'like', 'just', 'really', 'good', 'games', 'fun', 'story', 'world'], 0: ['frontiers', 'sonic', 'game', 'like', 'levels', 'open', 'games', 'just', 'cyberspace', 'world'], 1: ['denuvo', 'pc', 'support', 'fps', 'crashes', 'settings', 'steam', 'deck', 'fix', 'game'], 2: ['game', 'fun', 'story', 'recommend', 'good', 'great', 'really', 'feels', 'play', 'controls'], 3: ['like', 'sonic', 'game', 'just', 'stages', 'really', 'open', 'fun', 'feels', 'combat'], 4: ['sonic', 'poster', 'rides', 'puberty', 'tails', 'newspaper', 'bike', 'jumps', 'stock', 'appears'], 5: ['levels', 'cyberspace', 'cyber', 'open', 'fun', 'sonic', 'world', 'just', 'space', 'game'], 6: ['good', 'pretty', 'awesome', 'outstanding', 'alright', 'amazing', 'cool', 'fine', 'hoooooly', 'shetusdiopdkdjtettttttttttttttttttttttttttttttttt'], 7: ['flowing', 'face', 'cause', 'fear', 'hanging', 'won', 'wolves', 'know', 'll', 'light'], 8: ['3d', 'best', 'generations', 'sonic', 'finally', 'game', 'bay', 'stepped', '

In [19]:
dictionary.pop(-1)
print(dictionary)

{0: ['frontiers', 'sonic', 'game', 'like', 'levels', 'open', 'games', 'just', 'cyberspace', 'world'], 1: ['denuvo', 'pc', 'support', 'fps', 'crashes', 'settings', 'steam', 'deck', 'fix', 'game'], 2: ['game', 'fun', 'story', 'recommend', 'good', 'great', 'really', 'feels', 'play', 'controls'], 3: ['like', 'sonic', 'game', 'just', 'stages', 'really', 'open', 'fun', 'feels', 'combat'], 4: ['sonic', 'poster', 'rides', 'puberty', 'tails', 'newspaper', 'bike', 'jumps', 'stock', 'appears'], 5: ['levels', 'cyberspace', 'cyber', 'open', 'fun', 'sonic', 'world', 'just', 'space', 'game'], 6: ['good', 'pretty', 'awesome', 'outstanding', 'alright', 'amazing', 'cool', 'fine', 'hoooooly', 'shetusdiopdkdjtettttttttttttttttttttttttttttttttt'], 7: ['flowing', 'face', 'cause', 'fear', 'hanging', 'won', 'wolves', 'know', 'll', 'light'], 8: ['3d', 'best', 'generations', 'sonic', 'finally', 'game', 'bay', 'stepped', 'tracing', 'decade'], 9: ['pinball', 'pile', 'steaming', '10', 'oh', 'ass', 'sucks', 'buy', 

In [6]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

from torch import cuda

model_id = 'Qwen/Qwen2.5-1.5B-Instruct'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

print(device)

# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

cuda:0


In [7]:
def create_topic_prompt(topic_model: BERTopic, topic_id: int, data_string: list, max_docs: int = 5) -> str:
    """
    Convert a BERTopic topic into a prompt format for topic labeling using 
    the already trained model's data.
    
    Parameters:
    -----------
    topic_model : BERTopic
        Trained BERTopic model
    topic_id : int
        ID of the topic to convert
    data_string : list
        Original documents used for training
    max_docs : int, optional (default=5)
        Maximum number of documents to include in the prompt
        
    Returns:
    --------
    str
        Formatted prompt for topic labeling
    """
    # Get document info with the original documents
    doc_info = topic_model.get_document_info(data_string)
    
    # Filter for the specific topic and get top documents by probability
    topic_docs = doc_info[doc_info['Topic'] == topic_id]
    selected_docs = topic_docs.nlargest(max_docs, 'Probability')['Document'].tolist()
    
    # Format documents as a list
    formatted_docs = "\n".join(f"- {doc}" for doc in selected_docs)
    
    # Get keywords (top words) for the topic
    keywords = [word for word, _ in topic_model.get_topic(topic_id)]
    formatted_keywords = ", ".join(keywords)
    
    # Create the prompt
    prompt = f"""I have a topic that contains the following documents:
{formatted_docs}

The topic is described by the following keywords: '{formatted_keywords}'.

Based on the information about the topic above, please create a short label of this topic. Important : Make sure you to only return the label and nothing more."""
    
    return prompt

topic_id = 15  # Replace with your desired topic ID
prompt = create_topic_prompt(topic_model, topic_id, data_string, max_docs=5)

system_prompt_qwen2 = """<|im_start|>system
You are a helpful, respectful and honest assistant for labeling topics.<|im_end|>"""

example_prompt_qwen2 = """<|im_start|>user
"""+prompt+"""<|im_end|>"""

prompt_qwen2 = system_prompt_qwen2 + example_prompt_qwen2
res = generator(prompt_qwen2)
res_split = res[0]["generated_text"].replace(prompt_qwen2+"system", "")
print(res_split)


"Friendship Journey"


In [28]:
topic_id = 25

topic_documents = pd.DataFrame({'topic': topics, 'document': data_string})
find_topic = topic_documents[topic_documents.topic == topic_id]
find_topic.head()




,topic,document
48,25,The best 7/10 game you will ever play.\n\nIt's...
878,25,This game is alot better than i originally exp...
956,25,Cyberspace 1-2 man....brrr....
980,25,i honestly didn't think i'd enjoy this game to...
1290,25,Without a doubt the best game since generation...


In [29]:
prompt = create_topic_prompt(topic_model, topic_id, data_string, max_docs=5)

system_prompt_qwen2 = """<|im_start|>system
You are a helpful, respectful and honest assistant for labeling topics.<|im_end|>"""

example_prompt_qwen2 = """<|im_start|>user
"""+prompt+"""<|im_end|>"""

prompt_qwen2 = system_prompt_qwen2 + example_prompt_qwen2
res = generator(prompt_qwen2)
res_split = res[0]["generated_text"].replace(prompt_qwen2+"system", "")
def strip_first_line(s):
    lines = s.splitlines()
    lines.pop(0)
    return '\n'.join(lines)
label = strip_first_line(res_split)
print(label)

Label: Sonic & Knuckles: Cyber Space - Review


In [10]:
print(prompt_qwen2)

<|im_start|>system
You are a helpful, respectful and honest assistant for labeling topics.<|im_end|><|im_start|>user
I have a topic that contains the following documents:
- !! Sega please remove Denuvo !!

Denuvo forces you to have an internet connection or you can't play the game, and let's not forget the 24hr lockout.

Especially for Steam Deck and laptop users that do not always have internet access, it is the biggest problem with the game now. Will recommend the game once it is removed.
- Runs Good on my Ancient AMD FX-8350 on an Oldschool ASUS Sabertooth Motherboard, and 32G DDR3 Ram with an Nvidia 1660 6gig video Card.
- == WARNING! Denuvo is In this Game and is killing it! BEWARE ==


Sega, I love you but you made a stupid idea on having denuvo in this game, Its killing my fun here and I like to have fun!!! why do i need to be online all the time playing this, why am I getting locked out? and also, Why kill your modding support?!?!? They were why i wanted to play this on PC, are

In [11]:
topic_model.get_topic_info(16)

,Topic,Count,Name,Representation,Representative_Docs
0,16,51,16_baby_finally_sonic_bro,"[baby, finally, sonic, bro, childhoood, peakkk...","[sonic is back baby, SONIC IS BACK BABY!!!!!!!..."


In [12]:
topic_documents = pd.DataFrame({'topic': topics, 'document': data_string})
find_topic = topic_documents[topic_documents.topic == 16]
find_topic.head()

,topic,document
314,16,where were you when Sonic was good again?
422,16,Sonic May Cry 6
602,16,SONIC IS BACK
644,16,SONIC IS BACK\n\nSONIC IS FINALLY BACK
663,16,Sonic is awesome again


In [13]:
find_topic.head().to_numpy()

array([[16, 'where were you when Sonic was good again?'],
       [16, 'Sonic May Cry 6'],
       [16, 'SONIC IS BACK'],
       [16, 'SONIC IS BACK\n\nSONIC IS FINALLY BACK'],
       [16, 'Sonic is awesome again']], dtype=object)

In [14]:
topic_model.get_topic_info(16).to_numpy()

array([[16, 51, '16_baby_finally_sonic_bro',
        list(['baby', 'finally', 'sonic', 'bro', 'childhoood', 'peakkkkk', 'goattt', 'kingggggggggggggggggggggggggggggg', 'backkk', 'fames']),
        list(['sonic is back baby', 'SONIC IS BACK BABY!!!!!!!!', 'Sonic is back, baby!'])]],
      dtype=object)

In [15]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'frontiers | sonic | game | levels | like', 303],
                                   [1, 'pc | denuvo | refunded | fps | support', 261],
                                   [2, 'good | pretty | cool | awesome | outstanding', 89],
                                   ...,
                                   [78, 'cyberspace | island | boost | character | stages', 15],
                                   [79, 'sonic | games | gameplay | came | like', 15],
                                   [80, 'boss | final | legit | battles | fights', 15]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([303, 261,  89,  87,  77,  76,  73,  73,  73,  71,  71,  71,  60,  56,
                                         55,  51,  51,  50,  49,  45,  45,  44,  44,  42,  42,  41,  41,  40,
                                         38,  37,  36,  33,  32,  31,  31,  31,  30,  30,  30,  30,  29,  29,
                                         29,  29,  29,  28,  27,  27,  25,  24,  23,  23,  22,  22,  22,  21,
                                         21,  21,  21,  20,  20,  19,  19,  19,  19,  19,  18,  18,  18,  18,
                                         18,  18,  17,  16,  16,  16,  15,  15,  15,  15,  15]),
                         'sizemode': 'area',
                         'sizeref': np.float64(0.189375),
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([-4.063069  ,  3.1461198 , 11.277032  ,  4.31167   ,  7.5378256 ,
                          12.10036   , -7.498811  , 12.128624  , -2.849273  , -3.047445  ,
                           6.8175015 ,  7.599757  , -3.4120426 ,  3.7686377 , -2.3456817 ,
                          12.195676  ,  4.318493  , -3.1741111 , -7.442252  ,  6.936979  ,
                          11.806949  , 17.85703   , -3.9271505 , -2.4303372 ,  7.514371  ,
                          -2.5701604 , -3.2840726 ,  6.3312097 ,  6.8498588 ,  7.6219597 ,
                          11.244493  , 18.23007   , -2.4654465 , 11.455424  , -2.5687513 ,
                          -4.451928  , -2.1357641 ,  0.3101569 ,  7.1008496 ,  4.282837  ,
                           7.4934335 , 17.670692  , -4.201062  , 15.10589   , 11.732769  ,
                           3.9563525 , -4.040061  ,  3.8878653 ,  6.6531553 , 12.100457  ,
                          17.655972  ,  0.03145831, 11.4684305 , -2.603599  , -4.6338453 ,
                          -4.1411757 , 11.981446  , -7.753859  ,  0.03142054,  7.4252644 ,
                          11.334421  , -4.03153   , -3.1244304 , 15.105895  , -7.642437  ,
                           4.5550594 , 11.900504  ,  6.277408  , -3.9669766 ,  0.31021467,
                           7.4741607 , 11.39639   , 12.308437  , -2.4252455 ,  3.114391  ,
                          18.230076  ,  7.454047  , -3.7753956 , -2.6597323 , -3.9894626 ,
                          -3.1661983 ], dtype=float32),
              'xaxis': 'x',
              'y': array([  0.8377412 ,   0.3200389 ,  15.25835   ,  13.091885  ,   2.319698  ,
                          -11.662753  ,  15.729357  ,   6.767202  ,   1.0781143 ,   0.747505  ,
                            1.7343544 ,   2.4969175 ,  -0.96310776,  13.619549  ,  13.023109  ,
                            6.6592283 ,  13.085307  ,   0.38090256,  15.672855  ,   1.8055812 ,
                            6.25792   ,  18.28772   ,  -0.94645137,   0.28742546,  -4.740976  ,
                            1.3753753 ,  -0.92808074,   2.306823  ,   1.6504451 ,  -4.8483577 ,
                            6.003501  ,   0.11587118,  13.143